In [1]:
# Autenticar tu cuenta de Google
from google.colab import auth
auth.authenticate_user()

# Configurar las credenciales
from google.cloud import storage
client = storage.Client()


In [2]:
# Especifica el nombre del bucket y el archivo
bucket_name = 'etl_archivos'
file_name = 'yelp_reviews.csv'

# Descargar el archivo desde el bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)
blob.download_to_filename(file_name)

# Cargar el CSV en un DataFrame de pandas
import pandas as pd
reviews_df = pd.read_csv(file_name, on_bad_lines='skip') # Skip lines with errors using on_bad_lines

reviews_df.dropna(inplace=True)

<ipython-input-2-f3ef5f81251d>:12: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews_df = pd.read_csv(file_name, on_bad_lines='skip') # Skip lines with errors using on_bad_lines


In [3]:
import io
# bucket y archivo
bucket_name = 'etl_archivos'
file_name = 'business.csv'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df_by = pd.read_csv(io.BytesIO(file_content))

df_by['platform'] = 'Yelp'
df_by = df_by.drop(columns=['name','address','city','state','stars','review_count','categories'])

In [4]:
# prompt: left join reviews_df with df_by

df_join_yelp = pd.merge(reviews_df, df_by, on='business_id', how='left')
df_join_yelp = df_join_yelp.dropna(subset='latitude')
df_join_yelp['latitude'] = df_join_yelp['latitude'].astype(str)
df_join_yelp['longitude'] = df_join_yelp['longitude'].astype(str)
df_join_yelp['business_id'] = df_join_yelp['latitude'] + df_join_yelp['longitude']
df_join_yelp = df_join_yelp.drop(columns=['latitude','longitude'])
df_join_yelp.rename(columns={'stars': 'rating','platform':'platform_x'}, inplace=True)
df_join_yelp['date'] = pd.to_datetime(df_join_yelp['date'])
df_join_yelp['year'] = df_join_yelp['date'].dt.year
# Extraer el semestre
df_join_yelp['semester'] = (df_join_yelp['date'].dt.month - 1) // 6 + 1
df_join_yelp.head()


,business_id,date,rating,text,user_id,platform_x,year,semester
0,40.2101961875-75.2236385919,2018-07-07,3,"If you decide to eat here, just be aware it is...",mh_-eMZ6K5RLWhZyISBhwA,Yelp,2018,2
2,32.2072326-110.9808639,2014-02-05,3,Family diner. Had the buffet. Eclectic assortm...,8g_iMtfSiwikVnbP2etR0A,Yelp,2014,1
3,40.0798480557-75.025079772,2015-01-04,5,"Wow! Yummy, different, delicious. Our favo...",_7bHUi9Uuf5__HHc_Q8guQ,Yelp,2015,1
4,29.962102-90.087958,2017-01-14,4,Cute interior and owner (?) gave us tour of up...,bcjbaE6dDog4jkNY91ncLQ,Yelp,2017,1
5,39.9380132-75.1481307,2015-09-23,1,I am a long term frequent customer of this est...,eUta8W_HdHMXPzLBBZhL1A,Yelp,2015,2


In [5]:
# Create a StringIO object to hold the DataFrame in memory
sio = io.BytesIO()

# Write the DataFrame to the StringIO object in Parquet format
df_join_yelp.to_parquet(sio)

# Create a client object for interacting with Cloud Storage
client = storage.Client()

# Create a bucket object for the desired bucket
bucket = client.get_bucket('etl_archivos')

# Create a blob object for the desired file
blob = bucket.blob('ReviewsYelp.parquet')

# Upload the DataFrame from the StringIO object to the blob
blob.upload_from_string(sio.getvalue())

In [ ]:
# prompt: open reviewsyelp.parquet from cloud storage

from google.cloud import storage

# Specify the bucket and file names
bucket_name = 'etl_archivos'
file_name = 'ReviewsYelp.parquet'

# Create a storage client
storage_client = storage.Client()

# Get the bucket
bucket = storage_client.get_bucket(bucket_name)

# Get the blob (file)
blob = bucket.get_blob(file_name)

# Download the blob's contents to a BytesIO object
parquet_bytes = io.BytesIO(blob.download_as_string())

# Read the parquet file into a DataFrame using pandas
df_join_yelp = pd.read_parquet(parquet_bytes)
